In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ee
import urllib.request
import datetime
import os
import base64
import requests
import warnings
warnings.filterwarnings("ignore")
import tempfile
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload
from sentinelhub import SentinelHubRequest, MimeType, CRS, BBox, DataCollection
from sentinelhub import SHConfig
from sentinelhub import WmsRequest, CRS, MimeType, CustomUrlParam
from PIL import Image
import random

# Getting the Satellite Images

In [2]:
full_data = pd.read_csv('ODF_Fire_Occurrence_Data_2000-2022.csv')

In [3]:
full_data

,Serial,FireCategory,FireYear,Area,DistrictName,UnitName,FullFireNumber,FireName,Size_class,EstTotalAcres,...,Industrial_Restriction,Ign_DateTime,ReportDateTime,Discover_DateTime,Control_DateTime,CreationDate,ModifiedDate,DistrictCode,UnitCode,DistFireNumber
0,102649,STAT,2015,EOA,Klamath-Lake,Klamath,15-981082-16,Bass 497,B,3.20,...,Does Not Apply - Eastern OR,09/02/2015 05:00:00 PM,09/02/2015 05:05:00 PM,09/02/2015 05:00:00 PM,09/02/2015 11:00:00 PM,09/05/2015 12:00:00 AM,10/13/2015 08:39:00 AM,98,981,082
1,131239,STAT,2022,EOA,Walker Range - WRFPA,Crescent,22-991220-23,Hay Fire,A,NaN,...,NaN,NaN,08/16/2022 06:56:00 PM,NaN,NaN,08/18/2022 12:00:00 AM,08/18/2022 09:11:00 AM,99,991,220
2,58256,STAT,2000,EOA,Central Oregon,John Day,00-952011-01,Slick Ear #2,B,0.75,...,Does Not Apply - Eastern OR,07/18/2000 07:00:00 PM,07/19/2000 01:20:00 PM,07/19/2000 01:15:00 PM,07/20/2000 12:50:00 AM,07/20/2000 09:13:00 AM,11/14/2000 09:16:00 AM,95,952,011
3,59312,STAT,2000,EOA,Northeast Oregon,La Grande,00-971024-01,Woodley,C,80.00,...,Does Not Apply - Eastern OR,08/24/2000 05:30:00 AM,08/24/2000 01:07:00 PM,08/24/2000 01:07:00 PM,09/01/2000 09:30:00 PM,08/29/2000 03:59:00 PM,12/21/2000 04:22:00 PM,97,971,024
4,61657,STAT,2001,SOA,Southwest Oregon,Grants Pass,01-712133-02,QUEENS BRANCH,A,0.10,...,Lvl 3 Restricted Shutdown,08/10/2001 05:40:00 PM,08/10/2001 05:47:00 PM,08/10/2001 05:45:00 PM,08/10/2001 06:30:00 PM,08/10/2001 06:42:00 PM,08/17/2001 11:45:00 AM,71,712,133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23485,124548,STAT,2021,EOA,Walker Range - WRFPA,Crescent,21-991258-21,Road 2430,B,0.75,...,Does Not Apply - Eastern OR,06/26/2021 04:00:00 PM,06/26/2021 04:58:00 PM,06/26/2021 04:45:00 PM,06/26/2021 09:58:00 PM,07/01/2021 12:00:00 AM,10/02/2021 03:08:00 PM,99,991,258
23486,132141,STAT,2022,SOA,Western Lane,Veneta,22-781066-23,Spruce Path,A,0.01,...,Lvl 1 Fire Season Only,09/13/2022 11:20:00 AM,09/13/2022 11:20:00 AM,09/13/2022 11:20:00 AM,09/13/2022 12:14:00 PM,09/13/2022 12:00:00 AM,09/19/2022 10:09:00 AM,78,781,066
23487,124065,STAT,2021,EOA,Northeast Oregon,Pendleton,21-973052-21,Bone Canyon,C,67.43,...,Does Not Apply - Eastern OR,06/13/2021 05:22:00 PM,06/14/2021 12:45:00 PM,06/14/2021 12:45:00 PM,06/17/2021 05:10:00 PM,06/14/2021 12:00:00 AM,09/24/2021 01:48:00 PM,97,973,052
23488,131292,STAT,2022,EOA,Northeast Oregon,Pendleton,22-973014-23,Milepost 231,A,0.10,...,Does Not Apply - Eastern OR,08/19/2022 03:09:00 PM,08/19/2022 03:09:00 PM,08/19/2022 03:09:00 PM,08/19/2022 04:41:00 PM,08/19/2022 12:00:00 AM,11/08/2022 11:34:00 AM,97,973,014


In [4]:
# Collecting relevant columns
data = full_data[['FireYear','Size_class','FullFireNumber','EstTotalAcres','CauseBy','Lat_DD','Long_DD','FireName','ReportDateTime','Discover_DateTime','Control_DateTime','DistFireNumber']]

In [5]:
#Cleaning the data
data = data.dropna() # Drop rows with no discover time
data['Discover_DateTime'] = pd.to_datetime(data['Discover_DateTime']) # Convert to datetime 
data['Control_DateTime'] = pd.to_datetime(data['Control_DateTime']) # Convert to datetime 
data['ReportDateTime'] = pd.to_datetime(data['ReportDateTime']) # Convert to datetime 
data['Coordinates'] = data.apply(lambda row: [row['Lat_DD'], row['Long_DD']], axis=1) # Getting the coordinates

In [6]:
# Getting the dates for the images
data['6months'] = data['Discover_DateTime'] - pd.DateOffset(months=6)
data['3months'] = data['Discover_DateTime'] - pd.DateOffset(months=3)
data['1months'] = data['Discover_DateTime'] - pd.DateOffset(months=1)

In [7]:
test = data[data['FireYear'] >= 2016]

In [8]:
unique_names = data['FullFireNumber'].nunique()

In [9]:
unique_names

23189

## Getting the images

Use this to get access to the Earth Engine:
https://developers.google.com/earth-engine/guides/python_install

Username: forestfiresgroupglobaldtu

In [17]:
# Authenticate Earth Engine
#ee.Authenticate()

Enter verification code: 4/1AVHEtk77yGj5VfSb8qN-lJMZwVb7mI-gaQCJM33xpjipTTsO-bARsSmEx_M

Successfully saved authorization token.


## Earth Engine

In [38]:
# initialize Earth Engine
ee.Initialize()

### Google Drive Save

#### Landsat 8
Landsat 8, launched in February 2013, is an Earth observation satellite operated by the United States Geological Survey (USGS) and NASA. Landsat 8 operates in a sun-synchronous orbit, capturing images of the Earth's surface in a continuous and systematic manner.

Landsat 8 has a 16-day revisit time for any specific location on Earth. This means that Landsat 8 captures images of the same location approximately once every 16 days. The satellite acquires images in multiple spectral bands, including visible, near-infrared, and thermal infrared bands.

It's worth noting that Landsat 8 works in tandem with Landsat 9, which was launched in September 2021. Both satellites share the same orbital plane and follow the same ground track, with Landsat 9 offset by 8 days from Landsat 8. This effectively increases the revisit time for the Landsat program to approximately once every 8 days for any specific location on Earth.

In [20]:
# Function to get the image URL
def get_image_url(coordinates, timestamp):
    latitude, longitude = coordinates
    # convert the timestamp to a string in the format 'YYYY-MM-DD'
    date_str = timestamp.strftime('%Y-%m-%d')
    
    # convert the string to a datetime object
    date = datetime.datetime.strptime(date_str, '%Y-%m-%d')

    # Define the image collection and filter
    image_collection = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
        .filterDate(date - datetime.timedelta(days=30), date + datetime.timedelta(days=30)) \
        .filterBounds(ee.Geometry.Point(longitude, latitude))
    
    # Check if there are any images within the 30-day window
    image_count = image_collection.size().getInfo()
    if image_count == 0:
        #print(f'No images found within 30 days of {timestamp} for coordinates {coordinates}. Skipping row.')
        return
    
    # Get the least cloudy image
    least_cloudy = image_collection.sort('CLOUD_COVER').first()

    # Define the visualization parameters
    vis_params = {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0,
        'max': 3000,
        'gamma': 1.4
    }

    # Define the region to get the image
    region = ee.Geometry.Point(longitude, latitude).buffer(1000).bounds().getInfo()['coordinates']

    # Get the image URL
    image_url = least_cloudy.getThumbURL({
        'region': region,
        'scale': 12,  # Set the scale to match the native resolution
        'format': 'png',
        'resampling_method': 'bicubic',
        **vis_params
    })

    return image_url

# Function to save the image to Google Drive
def save_image_to_drive(image_url, file_name, folder_id=None):
    # Set up Google Drive API
    creds = service_account.Credentials.from_service_account_file('forestfiredtu-ef1cef2f2e43.json')
    service = build('drive', 'v3', credentials=creds)

    # Download the image
    response = requests.get(image_url)
    if response.status_code != 200:
        print(f"Failed to download image: {file_name}.png")
        return
    image_data = response.content

    # Save the image to a temporary file
    with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as temp_image:
        temp_image.write(image_data)
        temp_image.flush()

        # Save the image to Google Drive
        file_metadata = {
            'name': f'{file_name}.png',
            'mimeType': 'image/png'
        }

        if folder_id:
            file_metadata['parents'] = [folder_id]

        media = MediaFileUpload(temp_image.name, mimetype='image/png', resumable=True)

        try:
            file = service.files().create(body=file_metadata, media_body=media,
                                          fields='id').execute()
            print(F'File ID: "{file.get("id")}".')
        except HttpError as error:
            print(F'An error occurred: {error}')
            file = None

        # Remove the temporary file
        os.unlink(temp_image.name)

    return file
# Specify your Google Drive folder ID 
folder_id = '1As3zsmcQIIGIAwTdwgtPIcLQuLCclXsZ'

# Iterate over the DataFrame and call the functions
for index, row in df.iterrows():
    image_url = get_image_url(row['Coordinates'], row['6months'])
    if image_url:
        save_image_to_drive(image_url, row['FireName'], folder_id)


File ID: "1D_0XphtFY0jqAX5DEIyRWjP6pTxAUnA3".
File ID: "1wT2M-mDI7s3zRrTyHACLZQvrGdH2hSUG".
File ID: "1iWIZz_02_Jir7hoIpE4KAItqFC_YtOc_".
File ID: "1v1gvPIu8f8QGLhg_9Y_G490qo_HNsJvv".
File ID: "1VUX9paJhbWGYsIPm4lhinefqJ-GLWOEE".
File ID: "18S8sPyNPIxoBtKdDKxAe0KYOO6QPpZNX".
File ID: "1tTVldD5M6a1l0WFQJ99RrhkwT5OXbJpn".
File ID: "1hD86j5kD93QOj3yfd1D-0xdD7Oz-GtXu".
File ID: "1QBOdWyh-PMP5HiqGIQ58s_T2DDymVxG1".
File ID: "1oPhWrwfX-IQG6JDYGQ9UNCN42-BfPyiq".
File ID: "1d6Ng8cyRPAW8yKD7qUPPSyA8-14wvD6M".
File ID: "1rS2RZbFbaaUfBLiveIeEFBFolWHrBqJz".
File ID: "1E2vkNR8m55zSlzEQKp9JuNX65FW2Wwgc".
File ID: "1zhmPlei-8pACmcfEzrWnuTFCyxSwC9Nt".
File ID: "1RIM5nGkslI6WWoy3Zj7bhE9pjLGPgLsh".
File ID: "1bN7tEzwBzXCNA4oDDvyLScXYIgtcBgMl".
File ID: "14oqGf140uCtsryinf0vC17Ck7aLKV8Sc".
File ID: "1C9DWz38v2fZk3S5UL7-6ZOu-wyV-WVHG".
File ID: "15Wvgy3fR9iDWbDuhUB7vxDFk0lgq2SPR".
File ID: "1Uo3GNJcyimiZosuOL7ajtzfRW6fJ3TGr".
File ID: "1Z0WgUz0ZaAYHdq0-sOx-8k7EI2rEeWkv".
File ID: "1S29-bRlbYmWaP-ztnOGPPrM

#### Sentinel 2 - only data from 2015

In [30]:
# Function to get the image URL
def get_image_url(coordinates, timestamp):
    latitude, longitude = coordinates
    # convert the timestamp to a string in the format 'YYYY-MM-DD'
    date_str = timestamp.strftime('%Y-%m-%d')
    
    # convert the string to a datetime object
    date = datetime.datetime.strptime(date_str, '%Y-%m-%d')

    # Define the image collection and filter
    image_collection = ee.ImageCollection("COPERNICUS/S2") \
        .filterDate(date - datetime.timedelta(days=30), date + datetime.timedelta(days=30)) \
        .filterBounds(ee.Geometry.Point(longitude, latitude)) \
        .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 5)
    # Check if there are any images within the 30-day window
    image_count = image_collection.size().getInfo()
    if image_count == 0:
        #print(f'No images found within 30 days of {timestamp} for coordinates {coordinates}. Skipping row.')
        return

    # Get the least cloudy image
    least_cloudy = image_collection.sort('CLOUDY_PIXEL_PERCENTAGE').first()

    # Define the visualization parameters
    vis_params = {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0,
        'max': 3000,
        'gamma': 1.4
    }

    # Define the region to get the image
    region = ee.Geometry.Point(longitude, latitude).buffer(750).bounds().getInfo()['coordinates']

    # Get the image URL
    image_url = least_cloudy.getThumbURL({
        'region': region,
        'scale': 10,  # Set the scale to match the native resolution
        'format': 'png',
        'resampling_method': 'bicubic',
        **vis_params
    })

    return image_url

# Function to save the image to Google Drive
def save_image_to_drive(image_url, file_name, folder_id=None):
    # Set up Google Drive API
    creds = service_account.Credentials.from_service_account_file('forestfiredtu-ef1cef2f2e43.json')
    service = build('drive', 'v3', credentials=creds)

    # Download the image
    response = requests.get(image_url)
    if response.status_code != 200:
        print(f"Failed to download image: {file_name}.png")
        return
    image_data = response.content

    # Save the image to a temporary file
    with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as temp_image:
        temp_image.write(image_data)
        temp_image.flush()

        # Save the image to Google Drive
        file_metadata = {
            'name': f'{file_name}.png',
            'mimeType': 'image/png'
        }

        if folder_id:
            file_metadata['parents'] = [folder_id]

        media = MediaFileUpload(temp_image.name, mimetype='image/png', resumable=True)

        try:
            file = service.files().create(body=file_metadata, media_body=media,
                                          fields='id').execute()
            print(F'File ID: "{file.get("id")}".')
        except HttpError as error:
            print(F'An error occurred: {error}')
            file = None

        # Remove the temporary file
        os.unlink(temp_image.name)

    return file
# Specify your Google Drive folder ID 
folder_id = '13Yklp_2PlRh-HV1dmOMYA77L4fARfuIE'

# Iterate over the DataFrame and call the functions
for index, row in test.iterrows():
    image_url = get_image_url(row['Coordinates'], row['3months'])
    if image_url:
        save_image_to_drive(image_url, row['FireName'], folder_id)


File ID: "1a2fKCzTze1qxQXD_fpuECrF7FxZgCSLO".


In [43]:
# Function to get the image URL
def get_image_url(coordinates, timestamp):
    latitude, longitude = coordinates
    # convert the timestamp to a string in the format 'YYYY-MM-DD'
    date_str = timestamp.strftime('%Y-%m-%d')
    
    # convert the string to a datetime object
    date = datetime.datetime.strptime(date_str, '%Y-%m-%d')

    # Define the image collection and filter
    image_collection = ee.ImageCollection("COPERNICUS/S2") \
        .filterDate(date - datetime.timedelta(days=0), date + datetime.timedelta(days=35)) \
        .filterBounds(ee.Geometry.Point(longitude, latitude)) \
        .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 5)
    # Check if there are any images within the 30-day window
    image_count = image_collection.size().getInfo()
    if image_count == 0:
        #print(f'No images found within 30 days of {timestamp} for coordinates {coordinates}. Skipping row.')
        return

    # Get the least cloudy image
    least_cloudy = image_collection.sort('CLOUDY_PIXEL_PERCENTAGE').first()

    # Define the visualization parameters
    vis_params = {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0,
        'max': 3000,
        'gamma': 1.4
    }

    # Define the region to get the image
    region = ee.Geometry.Point(longitude, latitude).buffer(750).bounds().getInfo()['coordinates']

    # Get the image URL
    image_url = least_cloudy.getThumbURL({
        'region': region,
        'scale': 10,  # Set the scale to match the native resolution
        'format': 'png',
        'resampling_method': 'bicubic',
        **vis_params
    })

    return image_url

# Function to save the image to Google Drive
def save_image_to_drive(image_url, file_name, folder_id=None):
    # Set up Google Drive API
    creds = service_account.Credentials.from_service_account_file('forestfiredtu-ef1cef2f2e43.json')
    service = build('drive', 'v3', credentials=creds)

    # Download the image
    response = requests.get(image_url)
    if response.status_code != 200:
        print(f"Failed to download image: {file_name}.png")
        return
    image_data = response.content

    # Save the image to a temporary file
    with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as temp_image:
        temp_image.write(image_data)
        temp_image.flush()

        # Save the image to Google Drive
        file_metadata = {
            'name': f'{file_name}.png',
            'mimeType': 'image/png'
        }

        if folder_id:
            file_metadata['parents'] = [folder_id]

        media = MediaFileUpload(temp_image.name, mimetype='image/png', resumable=True)

        try:
            file = service.files().create(body=file_metadata, media_body=media,
                                          fields='id').execute()
            print(F'File ID: "{file.get("id")}".')
        except HttpError as error:
            print(F'An error occurred: {error}')
            file = None

        # Remove the temporary file
        os.unlink(temp_image.name)

    return file
# Specify your Google Drive folder ID 
folder_id = '13Yklp_2PlRh-HV1dmOMYA77L4fARfuIE'

# Iterate over the DataFrame and call the functions
for index, row in test.iterrows():
    image_url = get_image_url(row['Coordinates'], row['Control_DateTime'])
    if image_url:
        save_image_to_drive(image_url, row['FireName'], folder_id)


File ID: "1vtgAYvCL4xLBRsssJ8QWio54G5YfBsPO".


#### NAIP 100 rows
The National Agriculture Imagery Program (NAIP) imagery is typically updated on a two to three-year cycle. This means that for a specific area within the United States, new NAIP images are usually acquired every two or three years. However, this frequency may vary depending on factors like budget, weather conditions, and other factors that can impact aerial image acquisition.

NAIP primarily focuses on capturing imagery during the agricultural growing season (late spring through early fall) to support various agricultural programs and applications. The images have a resolution of 1 meter, and they are available in natural color (RGB) and, in some cases, near-infrared (NIR) bands.

In [24]:
#Split the data, as the scrape usually stops after 4000-5000 rows
data1 = data[:4000]
data2 = data[4000:8000]
data3 = data[8000:12000]
data4 = data[12000:16000]
data5 = data[16000:20000]
data6 = data[20000:]

In [19]:
# Function to get the image URL
def get_image_url(coordinates, timestamp):
    latitude, longitude = coordinates
    # convert the timestamp to a string in the format 'YYYY-MM-DD'
    date_str = timestamp.strftime('%Y-%m-%d')
    
    # convert the string to a datetime object
    date = datetime.datetime.strptime(date_str, '%Y-%m-%d')

    # Define the image collection and filter
    image_collection = ee.ImageCollection("USDA/NAIP/DOQQ") \
        .filterDate(date - datetime.timedelta(days=120), date + datetime.timedelta(days=30)) \
        .filterBounds(ee.Geometry.Point(longitude, latitude))
    
    # Check if there are any images within the 1-year window
    image_count = image_collection.size().getInfo()
    if image_count == 0:
        #print(f'No images found within 1 year of {timestamp} for coordinates {coordinates}. Skipping row.')
        return

    # Get the most recent image
    most_recent = image_collection.sort('system:time_start', False).first()

    # Define the visualization parameters
    vis_params = {
        'bands': ['R', 'G', 'B'],
        'min': 0,
        'max': 255,
    }

    # Define the region to get the image
    region = ee.Geometry.Point(longitude, latitude).buffer(250).bounds().getInfo()['coordinates']

    # Get the image URL
    image_url = most_recent.getThumbURL({
        'region': region,
        'scale': 1,  # Set the scale to match the native resolution
        'format': 'png',
        'resampling_method': 'bicubic',
        **vis_params
    })

    return image_url

# Function to save the image to Google Drive
def save_image_to_drive(image_url, file_name, folder_id=None):
    # Set up Google Drive API
    creds = service_account.Credentials.from_service_account_file('forestfiredtu-ef1cef2f2e43.json')
    service = build('drive', 'v3', credentials=creds)

    # Download the image
    response = requests.get(image_url)
    if response.status_code != 200:
        print(f"Failed to download image: {file_name}.png")
        return
    image_data = response.content

    # Save the image to a temporary file
    with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as temp_image:
        temp_image.write(image_data)
        temp_image.flush()

        # Save the image to Google Drive
        file_metadata = {
            'name': f'{file_name}.png',
            'mimeType': 'image/png'
        }

        if folder_id:
            file_metadata['parents'] = [folder_id]

        media = MediaFileUpload(temp_image.name, mimetype='image/png', resumable=True)

        try:
            file = service.files().create(body=file_metadata, media_body=media,
                                          fields='id').execute()
            print(F'File ID: "{file.get("id")}".')
        except HttpError as error:
            print(F'An error occurred: {error}')
            file = None

        # Remove the temporary file
        os.unlink(temp_image.name)

    return file
# Specify your Google Drive folder ID 
folder_id = '1ats_A6B5WJx2Gdkd426zOpVUSL6N_LEz'
j = 0
# Iterate over the DataFrame and call the functions
for index, row in data6.iterrows():
    image_url = get_image_url(row['Coordinates'], row['3months'])
    j += 1
    if j % 100 == 0:
        print(j)
    if image_url:
        save_image_to_drive(image_url, row['FullFireNumber'], folder_id)


File ID: "1qF3pMZr84DVCR_mm4obsEXwHuYiovdJP".
File ID: "14PE3z2Hi04sm5lre7v9i7oCv3Ni9Mvzg".
File ID: "1XyKdwpG0bYsTEK3mFKlpfepCwB_cC5x5".
File ID: "1F9Ya65s6ADBbEORz6s3kDjVpdt0qWfrm".
File ID: "1SQjla7YHxCZUublJ3qB9OCcfVeIK2MmT".
File ID: "1a1qiTOx2f52sJ-3dC7bE91WnvMiT1QFq".
File ID: "1y7rgsbU1HpNiyymm6MsMkg6vGffYr7aR".
File ID: "1m7k_AEnTrYcuxvDsvVWctyqo594lfzn4".
File ID: "1rAfZudhUNs45P5_X9sNKfN7cWWY5YEdV".
File ID: "10t0Dtfi64nBhdVAnzteU3Omi7EwDf18I".
File ID: "1f-teEgL5do1bsu7yGolF9_VcU6a0-7M5".
File ID: "1WQG9rp-NCC0OXK0vJC9Om2wfpP9x7FfH".
100
File ID: "15rMUwc7OSHgscrRpe_ILmUHAGTYBScJI".
File ID: "10230sm_Q1q6-W16TR-FFLxWJnmULgr78".
File ID: "1KSp1gOzt2LXg6rsRj1nTFQ4LeLfTLvFI".
File ID: "12JTgbSqBrHxquIY8a9l81fdIOB48ugTO".
File ID: "1I5-6J6Ivb1MVTmz6mYP8U43C_f0cItsn".
File ID: "1Hduz4JEjxzWxKK-rHtiLMe0kGM6pbXVE".
File ID: "1b8F9ldx60-A1cfSkjHJkRm3WYqQIZkmZ".
File ID: "1Ui2AS-M6WfTMdIH7F-eKPqWxLD_T5kTa".
File ID: "1ZLXmzJpHYHKOa_SFJwKruisBSSmppYKf".
File ID: "1BPkvMCTTPY5ruoTLJTo

File ID: "1YOJxvdg5Z2RZtOc6X_5beYD_Zv8zlwvd".
File ID: "1E3c3kLHzQ9RbhZigy3ikwBUm8vBHkXu6".
File ID: "1b_qKz9UQpmhr5kZXMTHx_YyRKnoTzWnn".
File ID: "12IMp1MjCkfaPAbELsLHNzjSEFGDKCs3p".
File ID: "1ci0ezqsjKp7oosSAasvneX9VfeCZoT-X".
File ID: "1VjH_WECBYJX4ZrrrTSZPtX53dyfvw9R1".
File ID: "1l-9voKQ2BCY5NLF17NOz3n4vtb2nVNrQ".
File ID: "1zi1R07Ybky4lMnewW1-KfmVctcZM5O7s".
1700
File ID: "12laswlmBdOyM5xdsqsRQ5X9ZeiPwqXlF".
File ID: "1X9a7QprA6TQwn8cWM6gej-PH5JrgmpXW".
File ID: "1rvSwqqCiwTLHzOUNC5w9vbHH--lBBmK0".
File ID: "1C4WBof5rL37beWn3bH8ssDBOSLIpUXM1".
File ID: "1trUFpkcIER9uEP_rrPAzmSN4E60doq-3".
File ID: "1Splv1LwHIHdNCqM0Y1ZpHNuJIbCpkV0F".
File ID: "1IiZBRB14h0qYqvo3R6FRpjNJcT7EN-iU".
File ID: "18RNRFFhJT8RbJFA0n7Ppor53FoB4WIoR".
File ID: "1aBBwc_18HusPh72UPOZGvsW29jXhzGlI".
1800
File ID: "1v1-MZI2dmo9YOt8hCICfW4BiTUdwSpMf".
File ID: "1kD_nuobJMZh0WOzVDSkUY5LWIux0a0JF".
File ID: "1x88WvmJfiN2nzCp8f4UmisZcGEGc2NJx".
1900
2000


KeyboardInterrupt: 

### Local Save

In [83]:
# Create a subfolder in the current folder
subfolder = 'images'
if not os.path.exists(subfolder):
    os.makedirs(subfolder)
    
# Function to get and save the image
def get_and_save_image(coordinates, timestamp, file_name):
    latitude, longitude = coordinates
    # convert the timestamp to a string in the format 'YYYY-MM-DD'
    date_str = timestamp.strftime('%Y-%m-%d')
    
    # convert the string to a datetime object
    date = datetime.datetime.strptime(date_str, '%Y-%m-%d')

    # Define the image collection and filter
    image_collection = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
        .filterDate(date - datetime.timedelta(days=30), date + datetime.timedelta(days=30)) \
        .filterBounds(ee.Geometry.Point(longitude, latitude))
    
    # Check if there are any images within the 30-day window
    image_count = image_collection.size().getInfo()
    if image_count == 0:
        #print(f'No images found within 30 days of {timestamp} for coordinates {coordinates}. Skipping row.')
        return
    
    # Get the least cloudy image
    least_cloudy = image_collection.sort('CLOUD_COVER').first()

    # Define the visualization parameters
    vis_params = {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0,
        'max': 3000,
        'gamma': 1.4
    }

    # Define the region to get the image
    region = ee.Geometry.Point(longitude, latitude).buffer(2500).bounds().getInfo()['coordinates']

    # Get the image URL
    image_url = least_cloudy.getThumbURL({
        'region': region,
        'dimensions': 512,
        'format': 'png',
        **vis_params
    })

    # Save the image
    # Save the image in the subfolder
    file_path = os.path.join(subfolder, f'{file_name}.png')
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content))
    img.save(file_path)
    print(f'Saved image: {file_path}')
# Iterate over the DataFrame and call the function
for index, row in df.iterrows():
    get_and_save_image(row['Coordinates'], row['6months'], row['FireName'])
    

Saved image: images/Bass 497.png
Saved image: images/Chilcoot.png
Saved image: images/CAZADERO TRAIL 6.png
Saved image: images/36 Pit.png
Saved image: images/White Creek.png
Saved image: images/White Mine Disp#553.png
Saved image: images/South Jetty.png
Saved image: images/Frosty Fire.png
Saved image: images/Pete's Camp.png
Saved image: images/MP 240 Hwy 101.png
Saved image: images/Bailey Hill Loop.png
Saved image: images/Sleppy Mine Rd.png
Saved image: images/Cheney Creek #2.png
Saved image: images/Redwood Hwy MP28.png
Saved image: images/Halls Creek.png
Saved image: images/Raven's Ridge Fire.png
Saved image: images/Sykes Creek.png
Saved image: images/N Old Stage Road.png
Saved image: images/Canary.png
Saved image: images/Trent Creek.png
Saved image: images/Big O.png
Saved image: images/Oak Knoll Rd.png
Saved image: images/Grizzly.png
Saved image: images/Johnson Canyon.png
Saved image: images/Kerby Mainline #2.png
Saved image: images/Grassy Ridge.png
Saved image: images/East King STAT

## Sentinal Hub

In [49]:
# Sentinel Hub Configuration
config = SHConfig()
# Replace 'your_sentinel_hub_api_key' with your actual API key
config.instance_id = "81b9baab-788a-4afb-9f4d-12edec4afadc"
config.sh_client_id = 'a8140661-00de-4dde-8c2d-1380ba95ac53'
config.sh_client_secret = ']vd@4d0RD|%2O4XgrLUTGj;o;D.zA,]-*pCu[.AY'

In [65]:
def get_image(coordinates, date, max_cloud_coverage=30):
    lat, lon = coordinates
    bbox = BBox(bbox=[lon - 0.025, lat - 0.025, lon + 0.025, lat + 0.025], crs=CRS.WGS84)
    # convert the timestamp to a string in the format 'YYYY-MM-DD'
    date = date.strftime('%Y-%m-%d')
    date_from = datetime.datetime.strptime(date, '%Y-%m-%d') - datetime.timedelta(days=70)
    date_to = datetime.datetime.strptime(date, '%Y-%m-%d') + datetime.timedelta(days=30)
    print(date_from)
    request = SentinelHubRequest(
        evalscript="""
            //VERSION=3
            function setup() {
                return {
                    input: [{
                        bands: ["B02", "B03", "B04", "SCL"]
                    }],
                    output: {
                        bands: 4,
                        sampleType: "UINT8"
                    }
                };
            }

            function evaluatePixel(sample) {
                return [
                    Math.max(0, Math.min(1, sample.B04 / 5000)) * 255,
                    Math.max(0, Math.min(1, sample.B03 / 5000)) * 255,
                    Math.max(0, Math.min(1, sample.B02 / 5000)) * 255,
                    sample.SCL
                ];
            }
        """,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L2A,
                time_interval=(date_from, date_to),
                maxcc=max_cloud_coverage / 100,
                mosaicking_order='leastCC'
            )
        ],
        responses=[
            SentinelHubRequest.output_response('default', MimeType.PNG)
        ],
        bbox=bbox,
        size=[512, 512],
        config=config
    )

    response = request.get_data()

    
    if not response:
        print(f"No images found for the specified date and location.")
        return None

    for index, image in enumerate(response):
        cloud_mask = image[..., 3]
        cloud_coverage = np.mean(cloud_mask == 8) * 100
       

        if cloud_coverage <= max_cloud_coverage:
            return image[..., :3]

    return None

def save_image_to_drive(image, file_name, folder_id=None):
    creds = service_account.Credentials.from_service_account_file('forestfiredtu-ef1cef2f2e43.json')
    service = build('drive', 'v3', credentials=creds)

    with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as temp_image:
        Image.fromarray(image).save(temp_image, 'PNG')
        temp_image.flush()

        file_metadata = {
            'name': f'{file_name}.png',
            'mimeType': 'image/png'
        }

        if folder_id:
            file_metadata['parents'] = [folder_id]

        media = MediaFileUpload(temp_image.name, mimetype='image/png', resumable=True)

        try:
            file = service.files().create(body=file_metadata, media_body=media,
                                          fields='id').execute()
            print(F'File ID: "{file.get("id")}".')
        except HttpError as error:
            print(F'An error occurred: {error}')
            file = None

        # Remove the temporary file
        os.unlink(temp_image.name)

    return file
                  
for index, row in df.iterrows():
    coordinates = tuple(row['Coordinates'])
    date = row['6months']
    image = get_image(coordinates, date)

    if image is not None:
        file_name = row['FireName']
        folder_id = '1m-xC9uM11r820r-9Pa8n0RGDWai1Iy6f'
        save_image_to_drive(image, file_name, folder_id)

2014-12-22 00:00:00
File ID: "1l3QFW1bUGTJ0qAHBnjKLVsUx7TL0IRmM".
1999-11-10 00:00:00
File ID: "1bFjhw3fjObwQjrwkYSnRLU_JiTSWrq9V".
1999-12-16 00:00:00
File ID: "1j0cNzZ9wMIs8f0F0XXU250ztv8qut2NY".
2000-12-02 00:00:00


KeyboardInterrupt: 

## Creating non fire dataset

In [51]:
# Function to generate a random timestamp between May and November of 2017-2021
def random_timestamp():
    year = random.choice(range(2017, 2022))
    month = random.choice(range(7, 12))
    day = random.choice(range(1, 30))
    return datetime.datetime(year, month, day)

# Function to generate random coordinates within Oregon
def random_coordinates():
    lat_min, lat_max = 42.0, 46.3
    lon_min, lon_max = -124.6, -116.5
    lat = random.uniform(lat_min, lat_max)
    lon = random.uniform(lon_min, lon_max)
    return [lat, lon]

# Function to generate a unique 9-digit ID number
def unique_id(existing_ids):
    id_num = random.randint(100000000, 999999999)
    while id_num in existing_ids:
        id_num = random.randint(100000000, 999999999)
    return id_num

# Generate dataset
timestamps = [random_timestamp() for _ in range(4000)]
coordinates = [random_coordinates() for _ in range(4000)]

ids = set()
unique_ids = []
for _ in range(4000):
    unique_id_num = unique_id(ids)
    ids.add(unique_id_num)
    unique_ids.append(unique_id_num)

data = {'Timestamp': timestamps, 'Coordinates': coordinates, 'ID Number': unique_ids}
df = pd.DataFrame(data)



In [52]:
df['3months'] = df['Timestamp'] - pd.DateOffset(months=3)

In [54]:
# Function to get the image URL
def get_image_url(coordinates, timestamp):
    latitude, longitude = coordinates
    # convert the timestamp to a string in the format 'YYYY-MM-DD'
    date_str = timestamp.strftime('%Y-%m-%d')
    
    # convert the string to a datetime object
    date = datetime.datetime.strptime(date_str, '%Y-%m-%d')

    # Define the image collection and filter
    image_collection = ee.ImageCollection("USDA/NAIP/DOQQ") \
        .filterDate(date - datetime.timedelta(days=120), date + datetime.timedelta(days=30)) \
        .filterBounds(ee.Geometry.Point(longitude, latitude))
    
    # Check if there are any images within the 1-year window
    image_count = image_collection.size().getInfo()
    if image_count == 0:
        #print(f'No images found within 1 year of {timestamp} for coordinates {coordinates}. Skipping row.')
        return

    # Get the most recent image
    most_recent = image_collection.sort('system:time_start', False).first()

    # Define the visualization parameters
    vis_params = {
        'bands': ['R', 'G', 'B'],
        'min': 0,
        'max': 255,
    }

    # Define the region to get the image
    region = ee.Geometry.Point(longitude, latitude).buffer(250).bounds().getInfo()['coordinates']

    # Get the image URL
    image_url = most_recent.getThumbURL({
        'region': region,
        'scale': 1,  # Set the scale to match the native resolution
        'format': 'png',
        'resampling_method': 'bicubic',
        **vis_params
    })

    return image_url

# Function to save the image to Google Drive
def save_image_to_drive(image_url, file_name, folder_id=None):
    # Set up Google Drive API
    creds = service_account.Credentials.from_service_account_file('forestfiredtu-ef1cef2f2e43.json')
    service = build('drive', 'v3', credentials=creds)

    # Download the image
    response = requests.get(image_url)
    if response.status_code != 200:
        print(f"Failed to download image: {file_name}.png")
        return
    image_data = response.content

    # Save the image to a temporary file
    with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as temp_image:
        temp_image.write(image_data)
        temp_image.flush()

        # Save the image to Google Drive
        file_metadata = {
            'name': f'{file_name}.png',
            'mimeType': 'image/png'
        }

        if folder_id:
            file_metadata['parents'] = [folder_id]

        media = MediaFileUpload(temp_image.name, mimetype='image/png', resumable=True)

        try:
            file = service.files().create(body=file_metadata, media_body=media,
                                          fields='id').execute()
            print(F'File ID: "{file.get("id")}".')
        except HttpError as error:
            print(F'An error occurred: {error}')
            file = None

        # Remove the temporary file
        os.unlink(temp_image.name)

    return file
# Specify your Google Drive folder ID 
folder_id = '1c0ECsmItewKnXzQVeLclpqekb4Vl5diN'
j = 0
# Iterate over the DataFrame and call the functions
for index, row in df.iterrows():
    image_url = get_image_url(row['Coordinates'], row['3months'])
    j += 1
    if j % 100 == 0:
        print(j)
    if image_url:
        save_image_to_drive(image_url, row['ID Number'], folder_id)


File ID: "1kHncnhepqXCTanHOKV6YoNJ_a8g-4zWu".
File ID: "1ByB8Xr20QJq9Mr2v29CO2anz6BBCthvr".
File ID: "116PuAUSY4aDrA8F-XOaKyRxE28V_GoXQ".
File ID: "1VT4hJ9VKP0tjTtZAnbGJQrbjYWmx7b95".
File ID: "19-kgknlUPG_75AN4mcsFsHgNJ7dJwo8x".
File ID: "1LrP-F26PADZi4ESGKlC2nXWnBEbgH8sD".
File ID: "1uf7V1C8-bvwW3IUKNaWWwxVHxTHcWkuD".
File ID: "1e9u8CzFaynDBZviu_vsB2fl_4VIxZu1A".
100
File ID: "1IbqyN-wEc5wbpIpYw2PA91GSphR3aFv7".
File ID: "1VecFJJiUHFpM9_hcI0izjloKmd8RTadB".
File ID: "14DaPhzJog-pgdxt98L0hkR4YIeOnNbzN".
File ID: "1lpcl1pVoecQg-tsQ3QDg3UfZBy52oe_3".
File ID: "1xpq3JVpx82LM9pT8uvHni1DWSL_Z8eey".
File ID: "1RktS2vfjSG_ZbsklMkX8RXpdoBjPqPUE".
File ID: "1aDy-jAWsr5x9y4pSQRa0RpABgOiXfx-a".
File ID: "1SPncrixvMbVfxXNRMWkbT8o1ahLrL-4s".
File ID: "1JQUigJACyuQd19uUtFXth5M5OC2_UH9n".
200
File ID: "1F3lUyXuP_EbSsZg-vhbjJEB2BufbBJ8O".
File ID: "1Uy14I1_jL6-L8cAcX8S1s-xr5ZqoBh8t".
File ID: "1M51jq-UlT3czus9DZ1b6YL0tIGD5yxxL".
File ID: "1-aSoqdbjbYSRqoUVDri3tgHmG_LeyOfP".
File ID: "1tm4HrSxD_Recrsm

File ID: "1FjZXOvSdYa6zJ6OwNjLvMxGIOQlVBVKA".
File ID: "1WEP60m_N6jFiwE-OTJUVGgoQKIE-_Kxc".
File ID: "11A_V_lk2MQsq1GNL679AyEDQgYU-I0O8".
File ID: "1PBV4YO112rDOWn8TzUVGmPFjdZ8zIquZ".
File ID: "1m0oDC4VxSRN3vuwSYdptc-HmrXeFvXsP".
File ID: "1eOMk6mxP9TeJwimnW_S-VZN8h0PLo3H8".
File ID: "1KoklZtFBihZNAI8uR9fy161d_OblnHYQ".
File ID: "19fnH8vjJnSqKdAKOWrNNZLzTMVUR1XsP".
1800
File ID: "1R2W64OaSqiCmHaqScgES75zYzqv5D7kv".
File ID: "12rDmt4dgGITDAH_so2PA83Tve2BQVOrd".
File ID: "1BSx5lHD9G0DMDTjtURBi4WjlQ79OUzhb".
File ID: "12j6W8adL8UJ584nElW-SDIX4IvgzODks".
File ID: "1B9Lb7i-KcjWpEBi_V0a3DzTk5ARP9dSx".
File ID: "1bicnKZErdeNh9R3GGQhXdlIs8FxOkudw".
File ID: "1nFOAoYJ1QDvzN-YM3d1Wtlvhv_-YTxJ_".
File ID: "1kweBFIjguYIPHUnhVdS8WLWGQW_U1Fwg".
1900
File ID: "1zYh2xwRfI-2qExMy6peK7OMRjoj-d9Iu".
File ID: "1tEzsSHJ0yjgml7LS4vfafC_gOtF7YNWL".
File ID: "1dRBcNKF4xLK3HuNGOky4kkyrFmfsSarr".
File ID: "1nWsOEWACUvuD0rqCkrndRbrp5hyL0qj6".
File ID: "1IFHF9iebz0lsnIw_PVLSAf3TfKX5k7g6".
File ID: "1aoHh1NeJQk_ts

3400
File ID: "1o98rB1k_kyjuMgqpccG9ChXeGtXkpu_D".
File ID: "1qeL4GkhrzmIg11Zy6fflgBXpa1kiAD8Z".
File ID: "1EdRqwxdbPrQ9zYUtonMzuxY92d4t3d-K".
File ID: "1AQtpLVpx_otpmtlZz8OFId_4J7draokb".
File ID: "1wSyDnopSK6hru2FCxSyjRzK7Hp0SEy6K".
File ID: "1Q6EB0Hv2gNqND1c7uj8CdSDoxIbmzEjl".
File ID: "1oOXg07zoSYGqFmpfigap4yV1draewln-".
File ID: "1ZrJs_t42yA-y0KaxakKCJ15vvxqEAe16".
File ID: "1rCfUL_PP7wGSLob0m5Dyhs_f56V6k9eL".
File ID: "1e0LE_VJpzoXfIOV-bNkGUUV74D3Ycr-r".
File ID: "1hg1RTKuh5ve1jdlQ3dMPz0C6egfwXpRA".
File ID: "1ZlxZfYaSsmiglRKtuQ8dgp1isKoO6NPN".
File ID: "1CO2ljK0WFAHt49UKAvs803d-icYYXOx8".
File ID: "1z-6c5vwo2iTQM1gdVFRNROo5QN5QzdF6".
File ID: "1ShLp0WNByE2Au9A_O0wUCjFrnfGFZno1".
3500
File ID: "1BLpy-H3EB4U4h7GoqYmXmViSKKOflcuu".
File ID: "1NxD6wDdyVCeAd6jCiw2hDryDcmXPQdd6".
File ID: "16GGgdePIVHH2nlPNNYosG0aNNuprUJ_W".
File ID: "1pKtllzixAhmTKgqleqXYs5efdGAFBnoX".
File ID: "1yoIy-_qgc6Xhrlv-Z7bFWtOizhqYlL6I".
3600
File ID: "1JS1cdEoo0JrW8cmQ5v2soQ_Bd5dZBttJ".
File ID: "1s9Eh_YBd